In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Flatten, Dense, BatchNormalization, Conv2D, MaxPooling2D, Dropout
import numpy as np
import pandas as pd
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import os
from PIL import Image

2023-03-30 19:54:56.771729: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 19:54:57.979041: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class BirdDataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, use_rows, batch_size, csv_file="birds.csv"):
        self.csv_df = pd.read_csv(csv_file)
        self.use_rows = use_rows
        self.batch_size = batch_size 
        self.dataset = pd.DataFrame(self.csv_df).to_numpy()
    
        self.labels = self.dataset[:,2]
        self.img_filepaths = self.dataset[:,1]
        self.class_ids = self.dataset[:,0]
        
        (x,) = self.img_filepaths.shape
        print(self.dataset.shape)
        print(self.img_filepaths[0])
        image = Image.open(self.img_filepaths[0])
        image_array = np.expand_dims(np.asarray(image), axis=0)
        
#         print(image_array)
        print(image_array.shape)
        
#         self.images = []
        
#         for i in range(0, 10):
#             image = Image.open(self.img_filepaths[0])
#             image_array = np.expand_dims(np.asarray(image), axis=0)
#             self.images.append(image_array)
            
#         self.images = np.array(self.images)
        
#         print(self.images.shape)
    
    def __len__(self):
        # batches_per_epoch is the total number of batches used for one epoch
        batches_per_epoch = int(len(self.use_rows) / self.batch_size)
        return batches_per_epoch
    
    def __getitem__(self, index):
        # index is the index of the batch to be retrieved
        batch_ids = self.use_rows[index * self.batch_size: (index + 1) * self.batch_size]
        
        x = None
        y = None
        
        images = []
        id_labels = []
        
        for curr_id in batch_ids:
            image = Image.open(self.img_filepaths[curr_id])
            img_array = np.asarray(image)
            images.append(img_array)
            id_labels.append(self.class_ids[curr_id])
#             print(self.class_ids[curr_id-1])
            
        
        images = np.array(images)
        id_labels = np.array(id_labels)
#         print(id_labels)
#         print(images.shape)
#         print(id_labels.shape)
#         print(self.class_ids)
        
#         print(self.class_ids.shape)
        
        x = images
        y = id_labels
        
        y = tf.keras.utils.to_categorical(y, num_classes=510)
#         print(x.shape)
#         print(y.shape)
        return x, y            
            
#     def on_epoch_end(self):
#         np.random.shuffle(self.split_ids)
            
            

In [3]:
use_rows = list(range(0,87050))
bird_data_generator = BirdDataGenerator(csv_file="birds.csv", use_rows=use_rows, batch_size=32)
bird_data_generator.__getitem__(2700)

(87050, 5)
train/ABBOTTS BABBLER/001.jpg
(1, 224, 224, 3)


(array([[[[ 80, 163, 167],
          [ 79, 162, 166],
          [ 82, 163, 167],
          ...,
          [ 38,  73,  75],
          [ 39,  73,  74],
          [ 39,  73,  74]],
 
         [[ 83, 164, 168],
          [ 83, 164, 168],
          [ 84, 165, 169],
          ...,
          [ 39,  74,  76],
          [ 40,  74,  75],
          [ 40,  74,  75]],
 
         [[ 84, 163, 168],
          [ 85, 164, 169],
          [ 87, 166, 171],
          ...,
          [ 37,  75,  76],
          [ 40,  76,  76],
          [ 41,  77,  77]],
 
         ...,
 
         [[ 48, 123, 126],
          [ 47, 122, 125],
          [ 50, 121, 125],
          ...,
          [ 61, 125, 125],
          [ 61, 125, 125],
          [ 60, 124, 124]],
 
         [[ 47, 122, 125],
          [ 47, 122, 125],
          [ 51, 122, 126],
          ...,
          [ 61, 125, 125],
          [ 63, 124, 125],
          [ 64, 125, 126]],
 
         [[ 47, 122, 125],
          [ 47, 122, 125],
          [ 53, 124, 128],
   

In [4]:
def bird_cnn_model(csv_file):
    model = Sequential([
        InputLayer(input_shape=(224,224,3)),
        Conv2D(filters=32, kernel_size=3, activation='relu'),
#         BatchNormalization(),
        MaxPooling2D(pool_size=2),
        Flatten(),
        Dense(510, activation="softmax")
    ])
    
    optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
    model.compile(loss="categorical_crossentropy", optimizer = optimizer, metrics=["accuracy"])
    
    print(model.summary())
    train_rows = list(range(0,81950))
    train_generator = BirdDataGenerator(csv_file=csv_file, use_rows=train_rows, batch_size=128)
    h = model.fit(x=train_generator, epochs=4, verbose=1)
    print(h)
    
#     validation_rows = list(range(84500, 87050))
#     validation_generator = BirdDataGenerator(csv_file=csv_file, use_rows=validation_rows, batch_size=128)
    
#     test_rows = list(range(81950, 84500))
#     test_generator = BirdDataGenerator(csv_file=csv_file, use_rows=test_rows, batch_size=128)
#     model.evaluate(x=test_generator)
    

In [5]:
cnn_model = bird_cnn_model("birds.csv")

2023-03-30 19:55:00.588995: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-30 19:55:00.683392: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-30 19:55:00.683908: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-30 19:55:00.688718: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-30 19:55:00.689258: I tensorflow/compile

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 394272)            0         
                                                                 
 dense (Dense)               (None, 510)               201079230 
                                                                 
Total params: 201,080,126
Trainable params: 201,080,126
Non-trainable params: 0
_________________________________________________________________
None
(87050, 5)
train/ABBOTTS BABBLER/001.jpg
(1, 224, 224, 3)
Epoch 1/4


2023-03-30 19:55:06.120361: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-03-30 19:55:12.253354: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-03-30 19:55:15.371459: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-30 19:55:15.563781: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 665.74MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-03-30 19:55:15.617654: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of 

 13/640 [..............................] - ETA: 15:08 - loss: 10248.9629 - accuracy: 0.0018

2023-03-30 19:55:36.113126: W tensorflow/core/framework/op_kernel.cc:1818] UNKNOWN: IndexError: index 510 is out of bounds for axis 1 with size 510
Traceback (most recent call last):

  File "/home/jer/miniconda3/envs/tf_env/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 267, in __call__
    ret = func(*args)

  File "/home/jer/miniconda3/envs/tf_env/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/jer/miniconda3/envs/tf_env/lib/python3.9/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/jer/miniconda3/envs/tf_env/lib/python3.9/site-packages/keras/engine/data_adapter.py", line 902, in wrapped_generator
    for data in generator_fn():

  File "/home/jer/miniconda3/envs/tf_env/lib/python3.9/site-packages/keras/engine/data_adapter.py", line 1049, in generato

UnknownError: Graph execution error:

2 root error(s) found.
  (0) UNKNOWN:  IndexError: index 510 is out of bounds for axis 1 with size 510
Traceback (most recent call last):

  File "/home/jer/miniconda3/envs/tf_env/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 267, in __call__
    ret = func(*args)

  File "/home/jer/miniconda3/envs/tf_env/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/jer/miniconda3/envs/tf_env/lib/python3.9/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/jer/miniconda3/envs/tf_env/lib/python3.9/site-packages/keras/engine/data_adapter.py", line 902, in wrapped_generator
    for data in generator_fn():

  File "/home/jer/miniconda3/envs/tf_env/lib/python3.9/site-packages/keras/engine/data_adapter.py", line 1049, in generator_fn
  